<a href="https://colab.research.google.com/github/parasgupta1995/Almabetter-Capstone-project/blob/main/Airline_Passenger_Referal_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



---


## ***The dataset is scarped in the spring 2019. Data contain within dataset provide information about whether a passenger will refer the airline to his/her friends based on their travel experience during the course of his/her journey with the airline.The features are listed below based on which passenger decide to refer the airline to his/her friends or not;***


---



---




In [124]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [125]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
df=pd.read_excel('/content/drive/MyDrive/DATA for projects/data_airline_reviews.xlsx')

In [127]:
df.shape

(131895, 17)

In [128]:
df.describe()

,overall,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money
count,64017.000000,60681.000000,60715.000000,52608.000000,44193.000000,39358.000000,63975.000000
mean,5.145430,2.952160,3.191814,2.908170,2.863372,2.692820,2.943962
std,3.477532,1.441362,1.565789,1.481893,1.507262,1.612215,1.587370
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000
50%,5.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,9.000000,4.000000,5.000000,4.000000,4.000000,4.000000,4.000000
max,10.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131895 entries, 0 to 131894
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   airline          65947 non-null  object 
 1   overall          64017 non-null  float64
 2   author           65947 non-null  object 
 3   review_date      65947 non-null  object 
 4   customer_review  65947 non-null  object 
 5   aircraft         19718 non-null  object 
 6   traveller_type   39755 non-null  object 
 7   cabin            63303 non-null  object 
 8   route            39726 non-null  object 
 9   date_flown       39633 non-null  object 
 10  seat_comfort     60681 non-null  float64
 11  cabin_service    60715 non-null  float64
 12  food_bev         52608 non-null  float64
 13  entertainment    44193 non-null  float64
 14  ground_service   39358 non-null  float64
 15  value_for_money  63975 non-null  float64
 16  recommended      64440 non-null  object 
dtypes: float64

In [130]:
df.head()

,airline,overall,author,review_date,customer_review,aircraft,traveller_type,cabin,route,date_flown,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,recommended
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Turkish Airlines,7.0,Christopher Hackley,8th May 2019,âœ… Trip Verified | London to Izmir via Istanb...,NaN,Business,Economy Class,London to Izmir via Istanbul,2019-05-01 00:00:00,4.0,5.0,4.0,4.0,2.0,4.0,yes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Turkish Airlines,2.0,Adriana Pisoi,7th May 2019,âœ… Trip Verified | Istanbul to Bucharest. We ...,NaN,Family Leisure,Economy Class,Istanbul to Bucharest,2019-05-01 00:00:00,4.0,1.0,1.0,1.0,1.0,1.0,no
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
# drop row which null value in recommended columns
df.dropna(axis=0,subset=['recommended'],inplace=True)

In [132]:
df.shape

(64440, 17)

In [133]:
# Converting the review date to datetime
def remove(x):
  sub_script=['st','nd','rd','th']

  for i in sub_script:
    x=x.replace(i,'')
  y=x.split(' ')
  y[1]=y[1][0:3]
  x=' '.join(y)
  return x
df['review_date']=pd.to_datetime(df['review_date'].apply(lambda x: remove(str(x))))
df['date_flown']=pd.to_datetime(df['date_flown'])

In [134]:
# Number of stop during the journey, if the df['route']==NaN then assumption is made that flight directly reaches the destination

def stop(x):
  '''Function to count the number of stop during the journey'''
  x=str(x).split(' ')
  count=0
  for i in x:
    i=i.lower()
    if i=='via':
      count+=1
  return count

df['stops']=df['route'].apply(lambda x: stop(x)) 


In [135]:
#Total passenger travel through the airline and there recommendation count whether a passenger refer the airline or not.

pass_rec=df[['airline','recommended','author']].groupby(by=['airline','recommended']).agg('count').reset_index()
pass_rec=pass_rec.pivot(index="airline",columns="recommended")
pass_rec.columns=['no','yes']
pass_rec['total passenger']=pass_rec.agg('sum',axis=1)
pass_rec=pass_rec.sort_values(by='total passenger').reset_index()
pass_rec

,airline,no,yes,total passenger
0,Thai Smile Airways,17,30,47
1,Tunisair,45,26,71
2,Air Arabia,58,22,80
3,Adria Airways,41,42,83
4,QantasLink,22,81,103
...,...,...,...,...
76,Cathay Pacific Airways,720,1632,2352
77,British Airways,1546,1177,2723
78,United Airlines,2177,593,2770
79,American Airlines,2216,560,2776


In [136]:
# Extraction review duration for each passenger and replacing NaN value by median.

df['review_duration(days)']=(df['review_date']-df['date_flown']).dt.days
df['review_duration(days)']=df['review_duration(days)'].fillna(df['review_duration(days)'].median())
df['review_duration(days)']

1          7.0
3          6.0
5          6.0
7         35.0
9          5.0
          ... 
131886    23.0
131888    23.0
131890    23.0
131892    23.0
131894    23.0
Name: review_duration(days), Length: 64440, dtype: float64

In [141]:
# Extracting the null value percentage for each columns

col=list(df.columns)
null_=[]

for i in col:
  count_null=df[df[i].isnull()==True][i].shape[0]
  total=df[i].shape[0]
  percent=round(count_null/total*100,2)
  null_.append(percent)
null_dict=dict(zip(col,null_))


In [139]:
# Imputing the NaN with mode for categorical data if percentage of NaN is less than 10%

print("The features which has NaN values less than 10% are follows:")
for i,col in enumerate(null_dict):
  if 0<null_dict.get(col)<10:
    np.where(df[col].isnull(),df[col].mode(),df[col])
    print('\n*',col)
  else:
    pass


The features which has NaN values less than 10% are follows:

* overall

* cabin

* seat_comfort

* cabin_service

* value_for_money


In [ ]:
# dropping column 'route','author','review_date','date_flown'

df.drop(['route','author','review_date','date_flown'],axis=1,inplace=True)